### DATA

需要分别读入face和pose的数据并对应起来
- 对于siblings_face 需要读入json  
- 对于siblings_pose 需要读入csv    
- 对于Label 需要读入csv


*特别注意对于face，需要去除置信度小于阈值的数据  
*要将video的label扩展到每个frame

In [1]:
import csv
import pandas as pd
import numpy as np
import json

In [2]:
import os
import pandas as pd

# 读取标签文件
label = pd.read_csv('labelA/回应情况-表格 1.csv')

In [3]:
label.head()


,切片ID,回应情况,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,强,中,弱,没有回应（忽视）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15YS_20230317_01/VCAM_0000,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15YS_20230317_01/VCAM_0000_1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15YS_20230317_01/VCAM_0000_2,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15YS_20230317_01/VCAM_0000_3,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
label.iloc[1,0][-4:]

'0000'

### 合并读取face和pose特征  
按照视频读取frame

In [4]:
def get_vel_and_acc(features):
    """
    cal the mean and var of velocity and acceleration respectively of input features
    """
    try:
        first_order_derivatives = np.gradient(features, axis=0)
        second_order_derivatives = np.gradient(first_order_derivatives, axis=0)
    except:
        print("Error in get_vel_and_acc")
        print(features.shape)
        print(print(f"len:{len(face_df0)} and {len(face_df1)}"))
        print(max_fm)
        print("------------------")

    # velecity
    vel_means = np.mean(first_order_derivatives, axis=0)
    vel_var = np.var(first_order_derivatives, axis=0)

    # acceleration
    acc_means = np.mean(second_order_derivatives, axis=0)
    acc_var = np.var(second_order_derivatives, axis=0)
    
    return vel_means, vel_var, acc_means, acc_var


In [6]:
# set openface confidence threshold
confidence_threshold = 0.1

# 导入特征并对齐标签
video_features = []
video_labels = []
labels = []
frame2video_id = [] # 记录frame属于哪个video
# 记录原本有多少条记录
face_count = 0
pose_count = 0
# 记录读入的记录
rface_count = 0
rpose_count = 0

# for each video
for i in range(1,len(label)):
    directory = label.iloc[i, 0]
    frame_features = []
    # VCAM number
    Date = directory.split('/')[0] # 15YS_20230317_01
    Vcam = directory.split('/')[-1]  # VCAM_xxxx_xx
    VcamID = Vcam.split('_')[1] # xxxx

    # reconstruct the correct Openpose directory
    video_directory = os.path.join('siblings_pose', Date,'VCAM_'+VcamID, Vcam)
    json_directory = os.path.join(video_directory, 'json')

    json_files = os.listdir(json_directory)
    json_files.sort(key=lambda x: int(x.split('_')[2])) # sort by the frame id
    

    # the correct Openface csv file
    feature_file = os.path.join('siblings_face', directory + '.csv')

    # the label data for this video
    label_value = label.iloc[i, 1:5] # 取1-4位的标签
    video_id = i # video id

    # Load face data of a video
    try:
        face_csv_data = pd.read_csv(feature_file)
    except:
        print(f"[face error]: {feature_file} not exist")
        continue
    
    # log the num of face data rows
    face_count += face_csv_data.shape[0]

    # 丢弃置信度小于0.5的数据
    face_csv_data = face_csv_data[face_csv_data['confidence'] >= confidence_threshold]

    # 找到frame中有多于2个不同face_id的帧
    counts = face_csv_data.groupby('frame')['face_id'].nunique()
    frames_to_drop = counts[counts > 2].index.tolist()

    
    # 对于每个需要处理的帧，删除置信度最低的行，直到只剩下2个face_id
    for frame in frames_to_drop:
        frame_df = face_csv_data[face_csv_data['frame'] == frame]
        while frame_df['face_id'].nunique() > 2:
            min_confidence = frame_df['confidence'].min()
            rows_to_drop = frame_df[(frame_df['confidence'] == min_confidence)]['face_id'].tolist()
            frame_df = frame_df[~frame_df['face_id'].isin(rows_to_drop)]
        face_csv_data = face_csv_data[face_csv_data['frame'] != frame]
        face_csv_data = pd.concat([face_csv_data, frame_df], ignore_index=True)

    # 丢弃只有1个face_id的帧 
    # TODO 是否可以不丢弃？
    face_csv_data = face_csv_data.groupby('frame').filter(lambda x: len(x) == 2)


    # 按照frame分组，并根据x_30(nose x位置)的大小对face_id进行赋值 x较小标记为0
    def assign_face_id(group):
        if len(group) != 2:
            return None
        else:
            if group['x_30'].iloc[0] < group['x_30'].iloc[1]:
                group['face_id'] = [0, 1]
            else:
                group['face_id'] = [1, 0]
            return group

    face_csv_data = face_csv_data.groupby('frame').apply(assign_face_id).reset_index(drop=True)
    # # 重新排序
    # face_csv_data= face_csv_data.set_index(['face_id', 'frame'])

    # def sort_by_frame(group):
    #     return group.sort_values(by='frame')

    # face_csv_data = face_csv_data.groupby('face_id').apply(sort_by_frame)


    # 将df根据face_id列分成两个DataFrame对象
    face_df0 = face_csv_data[face_csv_data['face_id'] == 0]
    face_df1 = face_csv_data[face_csv_data['face_id'] == 1]

    # 按照frame_id列进行排序
    # face_df0 = face_df0.sort_values(by='frame')
    # face_df1 = face_df1.sort_values(by='frame')




    # 将df的索引设置为frame_id,并且插值
    try:
        min_fm, max_fm = face_df0['frame'].min(), face_df0['frame'].max()
        if(max_fm - min_fm >2):
            # 记录contribute 数据数量
            rface_count += 2*face_df0.shape[0]
            # 插值
            face_df0 = face_df0.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
            face_df1 = face_df1.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
        else:
            print(f"[face error] in {feature_file}: not enough frames")
            # print(f"len:{len(face_df0)} and {len(face_df1)}")
            continue
    except:
        print(f"[face error] in {feature_file}:unknown error in interpolation and reindex")
        continue
    # # 对df进行插值
    #  face_df0.reset_index()
    
    ###############  face low feature #######################
    # low level feat to cal vel and acc
    face_low_feat = ['pose_Tx', 'pose_Ty', 'pose_Tz','pose_Rx', 'pose_Ry', 'pose_Rz', 'gaze_angle_x', 'gaze_angle_y']
    au_feat = ['AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU15_r',  'AU20_r', 'AU23_r', 'AU26_r']
    

    # 每个人的low feat 的 vel and acc
    v_mean_0, v_var_0, a_mean_0, a_var_0 = get_vel_and_acc(face_df0[face_low_feat])
    v_mean_1, v_var_1, a_mean_1, a_var_1 = get_vel_and_acc(face_df1[face_low_feat])
    au_mean_0 = np.array(face_df0[au_feat].mean())
    au_mean_1 = np.array(face_df1[au_feat].mean())  
    au_var_0 = np.array(face_df0[au_feat].var())
    au_var_1 = np.array(face_df1[au_feat].var())

    ###############  face high feature #######################
    # TODO face high feats


    # 每一frame的 face feature
    # features_face = face_csv_data.values 
    features_face_0 = np.concatenate((v_mean_0, v_var_0, a_mean_0, a_var_0,au_mean_0,au_var_0))
    features_face_1 = np.concatenate((v_mean_1, v_var_1, a_mean_1, a_var_1,au_mean_1,au_var_1))

    
    ###############  pose feature #######################
    
    
    pose_json_data = [] # save all frame data

    # feature names in 'pose_keypoints_2d' in json file
    col_name = [
        'frame','face_id',
       'x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 'x3', 'y3', 'c3', 'x4', 'y4', 'c4',
       'x5', 'y5', 'c5', 'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
       'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11', 'x12', 'y12', 'c12',
       'x13', 'y13', 'c13', 'x14', 'y14', 'c14', 'x15', 'y15', 'c15', 'x16', 'y16', 'c16', 
       'x17', 'y17', 'c17', 'x18', 'y18', 'c18', 'x19', 'y19', 'c19', 'x20', 'y20', 'c20', 
       'x21', 'y21', 'c21', 'x22', 'y22', 'c22', 'x23', 'y23', 'c23', 'x24', 'y24', 'c24'
       ]
    # 创建空的 dataframe
    pose_df0 = pd.DataFrame(columns=col_name)
    pose_df1 = pd.DataFrame(columns=col_name)
    
    
    # TODO del confidence as 
    # Load pose data of a video frame
    for file in json_files: # for a single frame
        file_path = os.path.join(json_directory, file)
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
        except:
            print(f"[pose error]: {file} not exist")
            continue

        # log the number
        pose_count += len(data['people'])
        
        # 漏检测 只有1人的情况
        if len(data['people']) != 2:
            # drop this frame
            continue
        
        row0 = data['people'][0]['pose_keypoints_2d'] # x0,y0,c0,x1,y1,c1,... x24,y24,c24
        row1 = data['people'][1]['pose_keypoints_2d']

        # face_id对应 x较小 标记为0，x较大 标记为1 
        if(row0[0]>row1[0]):
            row0,row1 = row1,row0 # swap

        # 插入face_id
        # TODO del face_id no need anymore
        row0.insert(0,0)
        row1.insert(0,1)

        # 插入frame
        frame_id = int(file.split('_')[-2]) # 文件 frame_id

        # 去掉多余的最后一 frame 文件 否者会导致 frame 重复
        if frame_id % 5 != 0:
            continue

        frame_id = frame_id // 5 + 1 # 恢复frame id 抽帧后的 id
        row0.insert(0,frame_id)
        row1.insert(0,frame_id)

        pose_df0.loc[len(pose_df0)] = row0
        pose_df1.loc[len(pose_df1)] = row1

        # pose_json_data.append(row0)
        # pose_json_data.append(row1)

    



     # 将df的索引设置为frame_id,并且插值
    try:
        # set frame to int type
        pose_df0['frame'] = pose_df0['frame'].astype(int)
        pose_df1['frame'] = pose_df1['frame'].astype(int)
        min_fm, max_fm = pose_df0['frame'].min(), pose_df0['frame'].max()

        if(max_fm - min_fm >2):
            # 记录contribute 数据数量
            rpose_count += 2*pose_df0.shape[0]
            # 插值
            pose_df0 = pose_df0.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
            pose_df1 = pose_df1.set_index('frame').reindex(range(min_fm, max_fm+1)).interpolate()
        else:
            print(f"[pose error] in {feature_file}: not enough frames")
            # print(f"len:{len(face_df0)} and {len(face_df1)}")
            continue
    except:
        print(f"[pose error] in {feature_file}:unknown error in interpolation and reindex")
        continue

    ###############  pose low feature #######################
    
    # TODO select low feat
    # pose_low_feat = ['x','y']
    pose_low_feat = col_name[2:]

    # 每个人的low feat 的 vel and acc
    # TODO something wrong with 450 feats
    v_mean_0, v_var_0, a_mean_0, a_var_0 = get_vel_and_acc(pose_df0[pose_low_feat])
    v_mean_1, v_var_1, a_mean_1, a_var_1 = get_vel_and_acc(pose_df1[pose_low_feat])
    mean_0 = np.array(pose_df0[pose_low_feat].mean())
    mean_1 = np.array(pose_df1[pose_low_feat].mean())
    var_0 = np.array(pose_df0[pose_low_feat].var())
    var_1 = np.array(pose_df1[pose_low_feat].var())
    

    ###############  pose high feature #######################
    # TODO pose high feature

    # left and right shoulder distance → (mean + var) = 2
    # hands-to-face distance (left, right) → (mean + var) = 4



    # features_pose = np.array(pose_json_data)
    features_pose_0 = np.concatenate((v_mean_0, v_var_0, a_mean_0, a_var_0,mean_0,var_0))
    features_pose_1 = np.concatenate((v_mean_1, v_var_1, a_mean_1, a_var_1,mean_1,var_1))

    # 将记录视频特征
    # video_features.append(np.array(frame_features))
    video_features.append(np.concatenate((features_face_0,features_pose_0,features_face_1,features_pose_1))) # simply attach them all together
    
    # video 级别label
    video_labels.append(np.argmax(label_value.astype("int"),axis=0)) # one-hot to class id
    
    # # frame级别label
    # video_labels.append(np.array(labels))

X = video_features
y = video_labels


# 总共遍历的face特征和pose特征数
print(f"Total face feat:{face_count}, pose feat:{pose_count}")
# 读入的数量
print(f"Read in videos X:{len(X)}, face_feat:{rface_count},pose_feat:{rpose_count}")
# # drop
print(f"Drop {face_count - rface_count} face features, {pose_count - rpose_count} pose features")


[face error]: siblings_face/15YS_20230317_01/VCAM_0000_3.csv not exist
[face error]: siblings_face/15YS_20230317_01/VCAM_0000_37.csv not exist
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_58.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_59.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_78.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_79.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_80.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_94.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_95.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_97.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_98.csv: not enough frames
[face error] in siblings_face/15YS_20230317_01/VCAM_0000_99.csv: not enough frames
[face error] in siblings_fa

In [6]:
print(f"Read in videos y:{len(y)}")

Read in videos y:544


维度

In [11]:
print(f"X:{len(X[0][0])}")

IndexError: index 0 is out of bounds for axis 0 with size 0

## Train

In [7]:
from sklearn.model_selection import train_test_split
# frame split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Video num: X_train:{len(X_train)}, X_test:{len(X_test)}, y_train:{len(y_train)}, y_test:{len(y_test)}")

Video num: X_train:435, X_test:109, y_train:435, y_test:109


Cross-validation  
- 5 folds

In [8]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, recall_score

def crossVal(rfc, X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

#     scoring = {'accuracy': make_scorer(accuracy_score),
#             'f1': make_scorer(f1_score, average='weighted'),
#             'recall': make_scorer(recall_score, average='weighted')}

    scoring = ['accuracy', 'f1_weighted', 'recall_weighted', 'precision_weighted']

    # 使用交叉验证器对模型进行评估
    scores = cross_validate(rfc, X, y, cv=kfold, scoring=scoring)

    # 输出交叉验证结果
    print('Accuracy:', scores['test_accuracy'].mean())
    print('F1 score:', scores['test_f1_weighted'].mean())
    print('Recall:', scores['test_recall_weighted'].mean())
    print('Precision:', scores['test_precision_weighted'].mean())


In [9]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, recall_score

def crossVal(rfc, X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

#     scoring = {'accuracy': make_scorer(accuracy_score),
#             'f1': make_scorer(f1_score, average='weighted'),
#             'recall': make_scorer(recall_score, average='weighted')}

    scoring = ['accuracy', 'f1_weighted', 'recall_weighted', 'precision_weighted']

    # 使用交叉验证器对模型进行评估
    scores = cross_validate(rfc, X, y, cv=kfold, scoring=scoring)

    # 输出交叉验证结果
    print('Accuracy:', scores['test_accuracy'].mean())
    print('F1 score:', scores['test_f1_weighted'].mean())
    print('Recall:', scores['test_recall_weighted'].mean())
    print('Precision:', scores['test_precision_weighted'].mean())


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 定义随机森林分类器模型
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# 定义交叉验证折数
K = 5

# 定义评估指标
scoring = ['accuracy', 'f1_weighted', 'recall_weighted', 'precision_weighted']

# 初始化评估指标结果列表
accuracy_scores = []
f1_scores = []
recall_scores = []
precision_scores = []

# 分割数据为K个折
data_folds = np.array_split(data, K)

# 对每个折进行交叉验证
for i in range(K):
    # 将第i个折作为验证集，其余折作为训练集
    data_train = np.concatenate(data_folds[:i] + data_folds[i+1:])
    data_test = data_folds[i]

    # 将训练数据和测试数据分开为X、y和labels
    X_train = data_train[:, :-2]
    y_train = data_train[:, -2]
    labels_train = data_train[:, -1]
    X_test = data_test[:, :-2]
    y_test = data_test[:, -2]
    labels_test = data_test[:, -1]

    # 训练模型
    rfc.fit(X_train, y_train)

    # 预测标签
    y_pred = rfc.predict(X_test)

    # 计算评估指标
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))

# 输出交叉验证结果
print('Accuracy:', np.mean(accuracy_scores))
print('F1 score:', np.mean(f1_scores))
print('Recall:', np.mean(recall_scores))
print('Precision:', np.mean(precision_scores))


In [11]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# 假设X是形状为(n_samples, num_frames, num_features)的特征数组
# 假设y是形状为(n_samples,)的标签数组
# 假设batch_size是每个batch包含的视频数
# 假设num_classes是标签的类别数
# 假设n_estimators是随机森林分类器的树数

# 获取数据的形状
n_samples, num_features = len(X), len(X[0][0])
num_frames = 0


# 计算batch数
num_batches = n_samples // batch_size


# 初始化分类器
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练分类器
for X_batch, y_batch in zip(X_batches, y_batches):
    # 将数据打平为二维数组
    X_flat = X_batch.reshape(-1, num_features)
    y_flat = np.repeat(y_batch, num_frames)

    # 训练分类器
    rfc.fit(X_flat, y_flat)

# 预测标签
y_pred = []
for X_batch in X_batches:
    # 将数据打平为二维数组
    X_flat = X_batch.reshape(-1, num_features)

    # 预测标签
    y_pred_batch = rfc.predict(X_flat)

    # 将预测结果重新打包成三维数组
    y_pred_batch = y_pred_batch.reshape(-1, num_frames)

    # 对每个视频的预测结果取众数
    y_pred_batch = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=y_pred_batch)

    # 将预测结果添加到y_pred列表中
    y_pred.append(y_pred_batch)

# 将预测结果合并为一维数组
y_pred = np.concatenate(y_pred)

# 计算评


TypeError: object of type 'numpy.float64' has no len()

### Random Forest

In [16]:

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# 使用随机森林预测一个frame
rfc = RandomForestClassifier(n_estimators=100, max_depth=10)
rfc.fit(X_train, y_train)
y_predict = rfc.predict(X_test)

print(f"acc:{rfc.score(X_test, y_test)}")
print(classification_report(y_test, y_predict, target_names=["强","中","弱","没有回应（忽视）"]))

acc:0.4036697247706422
              precision    recall  f1-score   support

           强       0.55      0.50      0.52        44
           中       0.25      0.07      0.11        14
           弱       0.27      0.60      0.37        20
    没有回应（忽视）       0.45      0.29      0.35        31

    accuracy                           0.40       109
   macro avg       0.38      0.37      0.34       109
weighted avg       0.43      0.40      0.39       109



In [17]:
crossVal(rfc,X,y)


/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.431855249745158
F1 score: 0.3939759228987903
Recall: 0.431855249745158
Precision: 0.4229085703859214


In [ ]:
# video test
import numpy as np

# 假设idx是一个形状为(n_videos,)的数组，保存了每个视频中的第一个帧的索引
# 假设y_pred和y_true分别是形状为(n_frames, n_classes)和形状为(n_videos,)的数组
# 其中y_pred保存了每个帧的预测结果，y_true保存了每个视频的真实标签

# 根据idx将y_pred的结果求和并取最大值作为视频级别的预测结果
video_pred = []
for i in idx:
    # 获取当前视频的帧的数量
    n_frames = np.sum(y_true == y_true[i])
    # 将当前视频的帧的预测结果相加并取平均
    video_result = np.mean(y_pred[i:i+n_frames], axis=0)
    # 将平均结果中的最大值作为视频级别的预测结果
    video_label = np.argmax(video_result)
    video_pred.append(video_label)

# 将y_true缩小到同样的数量，并与video_pred进行比较
y_true_reduced = y_true[idx]
accuracy = np.mean(y_true_reduced == video_pred)
print('Accuracy:', accuracy)


In [9]:
print(f"强：{len(y[y==0])}, 中：{len(y[y==1])}, 弱：{len(y[y==2])}, 没有回应（忽视）：{len(y[y==3])}")


强：5270, 中：2895, 弱：5147, 没有回应（忽视）：4361


In [6]:
len(frame2video_id)
# len(y)
# len(y_predict)

17673

In [ ]:
import numpy as np

# 假设X和y的形状分别为(n_frames, n_features)和(n_frames,)
# frame2video_id是一个形状为(n_frames,)的列表，保存了每个frame属于的视频编号
# 假设视频编号从1到N，其中N为视频数量



# 首先将y和frame2video_id转换为数组
y = np.array(y)
frame2video_id = np.array(frame2video_id)

print(N)

# 构建一个空数组，用于保存每个视频的分类结果
video_results = np.zeros((N, 4))

# 遍历每个视频编号
for i in range(1, N + 1):
    # 找到属于当前视频编号的帧的索引
    idx = np.where(frame2video_id == i)[0]
    print(idx)
    # 获取这些帧的分类结果
    frames = y_predict[idx]
    # 计算这些帧的平均分类结果
    video_result = np.mean(frames, axis=0)
    # 将平均分类结果保存到video_results数组中
    video_results[i - 1] = video_result

# 将video_results数组转换为整数编码的标签（即每个视频的类别）
video_labels = np.argmax(video_results, axis=1)


### DT

In [18]:
from sklearn.tree import DecisionTreeClassifier
# 建立决策树分类器
clf = DecisionTreeClassifier(random_state=0)
# clf.fit(X_train, y_train)
# y_predict = clf.predict(X_test)

# print(f"acc:{clf.score(X_test, y_test)}")
# print(classification_report(y_test, y_predict, target_names=["强","中","弱","没有回应（忽视）"]))

In [19]:
crossVal(clf,X,y)

Accuracy: 0.3160380564050289
F1 score: 0.31868009277261755
Recall: 0.3160380564050289
Precision: 0.33386054629195855


### KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

# 建立KNN分类器
clf = KNeighborsClassifier(n_neighbors=3)
# clf.fit(X_train, y_train)
# y_predict = clf.predict(X_test)

# print(f"acc:{clf.score(X_test, y_test)}")
# print(classification_report(y_test, y_predict, target_names=["强","中","弱","没有回应（忽视）"]))

In [21]:
crossVal(clf,X,y)

Accuracy: 0.30497791369351007
F1 score: 0.2889542019589667
Recall: 0.30497791369351007
Precision: 0.3191428030283897


### SVM
结果很差 看warning原因应该是有一些结果直接没有分类  
看看是否需要调参数或者直接去掉 

In [13]:
from sklearn.svm import SVC
# clf = SVC(kernel='linear')
clf = SVC()
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

print(f"acc:{clf.score(X_test, y_test)}")
print(classification_report(y_test, y_predict, target_names=["强","中","弱","没有回应（忽视）"]))

acc:0.4067892503536068
              precision    recall  f1-score   support

           强       0.39      0.59      0.47      1074
           中       0.00      0.00      0.00       559
           弱       0.41      0.35      0.38      1057
    没有回应（忽视）       0.43      0.51      0.47       845

    accuracy                           0.41      3535
   macro avg       0.31      0.36      0.33      3535
weighted avg       0.34      0.41      0.37      3535



/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [ ]:
crossVal(clf,X,y)

### XGBoost

In [14]:
import xgboost as xgb


# 定义模型参数
params = {
    'objective': 'multi:softmax',
    'num_class': 4,
    'eval_metric': 'mlogloss'
}

# 将数据转换为DMatrix格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# 训练模型
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

# 在测试集上预测
y_pred = model.predict(dtest)

# 计算准确率
accuracy = sum(y_pred == y_test) / len(y_test)
print('Accuracy:', accuracy)
print(classification_report(y_test, y_pred, target_names=["强","中","弱","没有回应（忽视）"]))


Accuracy: 0.3944954128440367
              precision    recall  f1-score   support

           强       0.60      0.41      0.49        44
           中       0.20      0.07      0.11        14
           弱       0.21      0.45      0.29        20
    没有回应（忽视）       0.48      0.48      0.48        31

    accuracy                           0.39       109
   macro avg       0.37      0.35      0.34       109
weighted avg       0.44      0.39      0.40       109



In [15]:
from xgboost import XGBClassifier

# 定义xgboost分类器模型
xgb = XGBClassifier(n_estimators=100, random_state=42)
crossVal(xgb,X,y)

Accuracy: 0.41904519198097184
F1 score: 0.4020817582525341
Recall: 0.41904519198097184
Precision: 0.414683914346187


### LSTM  
not working so far

In [77]:
from torch import nn
import torch

class LSTM(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc1 = nn.Linear(100, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 4) # 4 classes
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x) # 1 * 100
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out
        
class mil_regression(nn.Module):
    def __init__(self, input_size=786, hidden_size=100, num_layers=1, output_size=1):
        ''' use LSTM for MIL '''
        super(mil_regression, self).__init__()
        self.net = LSTM(input_size, hidden_size, num_layers)
        self.class_num = output_size

    def forward(self, inputs):
        # input shape: (frame_num, feature_size)
        

        self.seg_num, self.feature_num = inputs.shape #

        # outputs = torch.zeros((self.seg_num, self.class_num)).double.cuda() #  frame * 4（bool）

        outputs = self.net(inputs)
        # for i in range(self.seg_num):
        #     outputs[i,:] = self.net(inputs[i]) # 786

        # for idx, seg in enumerate(inputs):
        #     seg = Variable(seg).cuda()
        #     outputs[idx] = self.net(seg)

        # 视频特征 = frame取平均
        output = torch.mean(outputs, 1).cuda()
        return output

In [ ]:
import torch 

model = mil_regression().cuda()

epochs = 150
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    for i in range(len(X_train)): # 每次处理一个视频(对batch)

        x = torch.tensor(X_train[i]).float().cuda()
        y = torch.tensor(y_train[i]).float().cuda()
        

        if len(x.shape) ==1:
            continue

        optimizer.zero_grad()
        
        y_pred = model(x)

        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()
        
        # if (i+1) % 10 == 0:
        #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, len(X_train), single_loss.item()))

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

In [ ]:
with torch.no_grad():
    
    correct = 0
    total = 0
    for i in range(len(X_test)): # 每次处理一个视频(对batch)
        x = torch.tensor(X_test[i]).float().cuda()
        y = torch.tensor(y_test[i]).float().cuda()
        print(y)

        if len(x.shape) ==1:
            continue

        outputs = model(x)
        print(outputs)
        correct += (outputs == y)

### testing

In [10]:
import numpy as np

# 创建一个二维矩阵
x = np.array([[1, 2, 3], [4, 5, 6], [5, 8, 100],[5, 8, 100]])

# 计算梯度
dy = np.gradient(x)

# 输出结果
print(dy)


[array([[ 3. ,  3. ,  3. ],
       [ 2. ,  3. , 48.5],
       [ 0.5,  1.5, 47. ],
       [ 0. ,  0. ,  0. ]]), array([[ 1. ,  1. ,  1. ],
       [ 1. ,  1. ,  1. ],
       [ 3. , 47.5, 92. ],
       [ 3. , 47.5, 92. ]])]


In [9]:
import numpy as np

# 创建一个二维数组
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 计算每一行相邻行之间的二阶导数
d2x = []
for i in range(1, x.shape[0]-1):
    dx1 = np.gradient(x[i-1, :])
    dx2 = np.gradient(x[i, :])
    dx3 = np.gradient(x[i+1, :])
    d2x.append(np.diff(dx2-dx1) / np.diff(dx3-dx2))

# 输出结果
print(d2x)


[array([nan, nan])]


/tmp/ipykernel_162420/1742590338.py:12: RuntimeWarning: invalid value encountered in divide
  d2x.append(np.diff(dx2-dx1) / np.diff(dx3-dx2))


In [11]:
import numpy as np

# 创建一个二维数组
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 计算每一行数据对上一行数据的导数
dx = []
for i in range(1, x.shape[0]):
    diff = np.diff(x[i, :]) / np.diff(x[i-1, :])
    dx.append(diff)

# 输出结果
print(dx)


[array([1., 1.]), array([1., 1.])]


In [37]:
import pandas as pd

# 假设df是一个DataFrame对象，其中包含frame和feature两列
df = pd.DataFrame({'frame': [1, 3], 'feature': [1, 5]})

# 将df的索引设置为frame列，并增加需要插值的索引值
df = df.set_index('frame').reindex(range(df['frame'].min(), df['frame'].max()+1))

# 对df进行插值
df_interpolated = df.interpolate().reset_index()

# 输出插值结果
print(df_interpolated)


   frame  feature
0      1      1.0
1      2      3.0
2      3      5.0
